In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from typing import Dict, List
from dataclasses import dataclass
from langchain.messages import HumanMessage, AIMessage, SystemMessage

model = ChatOpenAI(model="gpt-5-mini", temperature=0, max_tokens=5000)


class Agent:
    def __init__(self, name: str, system_prompt: str, tools: list = None):
        self.name = name
        self.system_prompt = system_prompt
        self.tools = tools or []
        self.llm = model
        self.agent = create_agent(
            model=self.llm, tools=self.tools, system_prompt=self.system_prompt
        )
        self.conversations: Dict[str, List] = {}

    def intial_message(
        self, agent: Agent, system_message: str, max_turns: int = 5
    ) -> str:
        self.conversations[agent.name] = [
            SystemMessage(content=self.system_prompt),
            SystemMessage(content=system_message),
        ]

        conversation = self.agent.invoke({"messages": self.conversations[agent.name]})

        initial_message = conversation["messages"][-1]

        self.conversations[agent.name].append(initial_message)

        agent.respond(self, initial_message.content, max_turns=max_turns)

        print(f"{self.name} to {agent.name}: {initial_message.content}")

        return initial_message.content

    def respond(self, agent: Agent, content: str, max_turns: int) -> str:
        if (
            len(
                [
                    message
                    for message in self.conversations.get(agent.name, [])
                    if not isinstance(message, SystemMessage)
                ]
            )
            >= max_turns * 2
        ):
            print(
                f"{self.name} has reached the maximum number of turns with {agent.name}."
            )
            return "Conversation ended due to maximum turns reached."

        if agent.name not in self.conversations:
            self.conversations[agent.name] = []
        self.conversations[agent.name].append(HumanMessage(content=content))

        conversation = self.agent.invoke({"messages": self.conversations[agent.name]})
        response_message = conversation["messages"][-1]

        self.conversations[agent.name].append(response_message)

        print(f"{self.name} to {agent.name}: {response_message.content}")

        if END_PHRASE in response_message.content.lower():
            print(f"{self.name} ended the conversation with {agent.name}.")
            return response_message.content

        agent.respond(self, response_message.content, max_turns=max_turns)

        return response_message.content


# @tool
def pov(agent_name: str) -> str:
    """Returns the point of view for a given name."""
    povs = {
        "Alice": "",
        "Bob": """You absolutely cannot be friends or in any relationship with someone who is whose name is "Alice". Immediately end the conversation with such people.""",
    }
    print(f"Retrieving POV for agent {agent_name}")
    return povs.get(agent_name, f"Point of view for {agent_name} is not available.")


END_PHRASE = "goodbye"

SYSTEM_PROMPT = """Your name is {name} You attend a speed dating event.
You will talk to other people and your job is to find a compatible partner.

# Beliefs and Point of View
You must strictly adhere to your point of view. Use the tool `pov`.
This is your point of view: {pov_response}

# Conversation stop:
At any point in the conversation you may choose to stop the conversation if you want. Say the word "{end_phrase}" to end the conversation.
"""

INITIALIZE_CONVERSATION_PROMPT = """Please start a conversation with {}."""


agent1 = Agent(
    name="Alice",
    system_prompt=SYSTEM_PROMPT.format(
        name="Alice", end_phrase=END_PHRASE, pov_response=pov(agent_name="Alice")
    ),
    # tools=[pov],
)

agent2 = Agent(
    name="Bob",
    system_prompt=SYSTEM_PROMPT.format(
        name="Bob", end_phrase=END_PHRASE, pov_response=pov(agent_name="Bob")
    ),
    # tools=[pov],
)

initial_message = agent1.intial_message(
    agent=agent2,
    system_message=INITIALIZE_CONVERSATION_PROMPT.format(agent2.name),
    max_turns=2,
)
agent1.conversations[agent2.name]